# Cloud Inpainting on SEN3 Data
This notebook is a quick test to load, mask with clouds and then try and recover the data hidden by clouds using an inpainting DL model

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from network.model import SAGAN
import keras

%load_ext autoreload

%autoreload 2


2024-06-28 09:03:15.000260: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-28 09:03:15.000777: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-28 09:03:15.004459: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-28 09:03:15.051024: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-28 09:03:15.836884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

## Extracting the bands from the Xarrays

The goal here is to convert the data into a more tensorflow friendly data format

In [3]:
dsnp_c = np.moveaxis(np.load("Cloudy_DS2.npy"),3,0)
dsnp = np.moveaxis(np.load("Ground_truth2.npy"),3,0)

In [4]:
#Data normalisation

"""for i in range(dsnp.shape[3]):
    for j in range(11):
        dsnp[:,:,i,j] = (dsnp[:,:,i,j] - np.min(dsnp[:,:,i,j]))/(np.max(dsnp[:,:,i,j]) - np.min(dsnp[:,:,i,j]))
        dsnp_c[:,:,i,j] = (dsnp_c[:,:,i,j] - np.min(dsnp_c[:,:,i,j]))/(np.max(dsnp_c[:,:,i,j]) - np.min(dsnp_c[:,:,i,j]))"""

'for i in range(dsnp.shape[3]):\n    for j in range(11):\n        dsnp[:,:,i,j] = (dsnp[:,:,i,j] - np.min(dsnp[:,:,i,j]))/(np.max(dsnp[:,:,i,j]) - np.min(dsnp[:,:,i,j]))\n        dsnp_c[:,:,i,j] = (dsnp_c[:,:,i,j] - np.min(dsnp_c[:,:,i,j]))/(np.max(dsnp_c[:,:,i,j]) - np.min(dsnp_c[:,:,i,j]))'

In [5]:
#Split into training and test set


train_ds = tf.data.Dataset.from_tensor_slices((np.stack((dsnp_c[:800,:,:,3],dsnp_c[:800,:,:,-1]), axis=-1), dsnp[:800,:,:,3]))
test_ds = tf.data.Dataset.from_tensor_slices((np.stack((dsnp_c[800:,:,:,3], dsnp_c[800:,:,:,-1]),axis=-1), dsnp[800:,:,:,3]))

In [6]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 800

train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).cache()
test_ds = test_ds.batch(BATCH_SIZE)

## Model Definition

In [2]:
model = SAGAN()
gen_opti = keras.optimizers.Adam(learning_rate=0.0001)
disc_opti = keras.optimizers.Adam(learning_rate=4*0.0001)
model.compile(strategy=None, gen_optimizer=gen_opti, dis_optimizer=disc_opti)
print(model.summary())


/home/etienne/.local/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/home/etienne/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'gated_conv2d_14', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


x (None, 64, 64, 128)
proj_query (None, 4096, 16)
proj_key (None, 16, 4096)
proj_value (None, 128, 4096)
attention (None, 4096, 4096)


/home/etienne/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'gated_conv2d_31', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/home/etienne/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


x (None, 4, 4, 256)
proj_query (None, 16, 32)
proj_key (None, 32, 16)
proj_value (None, 256, 16)
attention (None, 16, 16)


Model: "sagan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gen_model (GenModel)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ self_attention_discriminator    │ ?                      │   0 (unbuilt) │
│ (SelfAttentionDiscriminator)    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_6 (Functional)       │ ?                      │    14,714,688 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,718,574 (98.11 MB)

 Trainable params: 10,996,330 (41.95 MB)

 Non-trainable params: 14,722,244 (56.16 MB)

None


In [ ]:
#np.max(dsnp_c[:,:,:,0])
out = model.predict(dsnp_c[10:11,:,:,1:3])[0,:,:,:]



#np.max(out)
#normalise prediction
#for i in range(11):
out2 = (np.log(out[:,:,:]) - np.min(np.log(out[:,:,:])))/(np.max(np.log(out[:,:,:])) - np.min(np.log(out[:,:,:])))

In [ ]:
plt.subplot(1,3,1)
plt.imshow(dsnp[10,:,:,1])
plt.subplot(1,3,2)
plt.imshow(dsnp_c[10,:,:,1])
plt.subplot(1,3,3)
plt.imshow(out[:,:,0])


In [ ]:
print(model.discriminator.summary())

In [7]:
model.fit(train_ds, epochs=1, validation_data=test_ds, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True), keras.callbacks.ModelCheckpoint(filepath='coarseNet.keras', save_best_only=True, save_freq="epoch"), keras.callbacks.TensorBoard(log_dir='./logs'), keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0), keras.callbacks.CSVLogger('coarseNet.csv'), keras.callbacks.TerminateOnNaN()])


x (None, 64, 64, 128)
proj_query (None, 4096, 16)
proj_key (None, 16, 4096)
proj_value (None, 128, 4096)
attention (None, 4096, 4096)
x (None, 4, 4, 256)
proj_query (None, 16, 32)
proj_key (None, 32, 16)
proj_value (None, 256, 16)
attention (None, 16, 16)
x (None, 64, 64, 128)
proj_query (None, 4096, 16)
proj_key (None, 16, 4096)
proj_value (None, 128, 4096)
attention (None, 4096, 4096)
x (None, 4, 4, 256)
proj_query (None, 16, 32)
proj_key (None, 32, 16)
proj_value (None, 256, 16)
attention (None, 16, 16)


2024-06-28 09:04:09.552126: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] remapper failed: INVALID_ARGUMENT: Mutation::Apply error: fanout 'StatefulPartitionedCall/gradient_tape/gen_model_1/sequential_1/gated_de_conv2d_1/leaky_re_lu_12_1/LeakyRelu/LeakyReluGrad_1' exist for missing node 'StatefulPartitionedCall/gen_model_1/sequential_1/gated_de_conv2d_1/conv2d_24_1/add'.
2024-06-28 09:04:10.542168: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
2024-06-28 09:04:32.457983: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [64,256,256,1] vs. [64,256,256]
	 [[{{function_node __inference_one_step_on_data_51828}}{{node gradient_tape/sub_6/BroadcastGradientArgs}}]]


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/sub_6/BroadcastGradientArgs defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/etienne/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/etienne/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/etienne/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/etienne/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/etienne/.local/lib/python3.10/site-packages/ipyke

In [ ]:
validation_loss = model.evaluate(test_ds)

In [ ]:
#Load keras file 


model.load_weights("./tuner/coarseNet_tuning/trial_0000/checkpoint.weights.h5")

In [ ]:
extractor = keras.Model(inputs=model.model.inputs, outputs=[layer.output for layer in model.model.layers])
features = extractor.predict(dsnp_c[1:2,:,:,1:3])  


fig = plt.figure()
for i, feature in enumerate(features):
    ax = fig.add_subplot(5,4,i+1)
    ax.imshow(feature[0,:,:,0])
fig.savefig('features-2.png')